In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img
#import matplotlib.pyplot as plt
import pathlib
import os
import shutil
import random

In [2]:
baseDir = "/Users/lukas/GitHub/rescue_maze_2024/vision"
victim_root = os.path.join(baseDir, "victims")

sourceDir = os.path.join(baseDir, "source")

trainingDir = os.path.join(victim_root,"training")
validationDir = os.path.join(victim_root,"validation")



if os.path.exists(victim_root):
  shutil.rmtree(victim_root)


def create_dirs(victim_root):
  os.mkdir(victim_root)
  level_one_dir = ["validation","training"]
  level_two_dir = ["H","U","S", "none"]

  for dir in level_one_dir:
    path = os.path.join(victim_root, dir)
    os.mkdir(path)
    for dir2 in level_two_dir:
        path2 = os.path.join(path, dir2)
        os.mkdir(path2)


create_dirs(victim_root)

for rootdir, dirs, files in os.walk(victim_root):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))







/Users/lukas/GitHub/rescue_maze_2024/vision/victims/training
/Users/lukas/GitHub/rescue_maze_2024/vision/victims/validation
/Users/lukas/GitHub/rescue_maze_2024/vision/victims/training/U
/Users/lukas/GitHub/rescue_maze_2024/vision/victims/training/S
/Users/lukas/GitHub/rescue_maze_2024/vision/victims/training/H
/Users/lukas/GitHub/rescue_maze_2024/vision/victims/training/none
/Users/lukas/GitHub/rescue_maze_2024/vision/victims/validation/U
/Users/lukas/GitHub/rescue_maze_2024/vision/victims/validation/S
/Users/lukas/GitHub/rescue_maze_2024/vision/victims/validation/H
/Users/lukas/GitHub/rescue_maze_2024/vision/victims/validation/none


In [3]:
def split_data(SOURCE_DIR_r, TRAINING_DIR_r, VALIDATION_DIR_r, SPLIT_SIZE):   
    sub_dir = ["H","S","U", "none"]
    for dir in sub_dir:
        print(dir)
        SOURCE_DIR = os.path.join(SOURCE_DIR_r,dir)
        TRAINING_DIR = os.path.join(TRAINING_DIR_r,dir)
        VALIDATION_DIR = os.path.join(VALIDATION_DIR_r,dir)
        print(SOURCE_DIR)
        list = os.listdir(SOURCE_DIR)


        random.sample(list, len(list))

        list2 = []
        for i in range(len(list)):
            source = os.path.join(SOURCE_DIR, list[i])
            if os.path.getsize(source) == 0:
                print(f"{list[i]} is zero length, so ignoring.")
            else:
                list2.append(list[i])
        print(len(list))
        #print(list)
        for i in range(len(list2)):

            source = os.path.join(SOURCE_DIR, list2[i])
            if i < len(list2)*SPLIT_SIZE:
                destination = os.path.join(TRAINING_DIR, list2[i])
            else:
                destination = os.path.join(VALIDATION_DIR, list2[i])
            #print(destination)
            shutil.copyfile(source, destination)


In [4]:

sourceDir = os.path.join(baseDir, "source")
sourceList = os.listdir(sourceDir)
trainingDir = os.path.join(victim_root,"training")
validationDir = os.path.join(victim_root,"validation")

    


split_data(sourceDir, trainingDir, validationDir, 0.9)

H
/Users/lukas/GitHub/rescue_maze_2024/vision/source/H
546
S
/Users/lukas/GitHub/rescue_maze_2024/vision/source/S
1352
U
/Users/lukas/GitHub/rescue_maze_2024/vision/source/U
433
none
/Users/lukas/GitHub/rescue_maze_2024/vision/source/none
0


In [10]:
inputShape = (150,150)
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
    global trainGenerator, validationGenerator
    train_datagen = ImageDataGenerator(rescale=1./255)
    validation_datagen = ImageDataGenerator(rescale=1./255)

    trainGenerator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                       target_size= inputShape, 
                                                       class_mode="categorical", 
                                                       batch_size = 128

                                                       )
    validationGenerator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                                target_size=inputShape, 
                                                                class_mode="categorical"
                                                                 )
train_val_generators(trainingDir,validationDir)

Found 2099 images belonging to 4 classes.
Found 232 images belonging to 4 classes.


In [11]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=inputShape),
    tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation="relu"),
  tf.keras.layers.Dense(3, activation="softmax")
])

model.summary()
model.compile(optimizer="adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


ValueError: Input 0 of layer "conv2d_4" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 150, 150)

In [7]:
model.fit(trainGenerator, epochs = 10, validation_data =validationGenerator)    
                                                                 

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sequential/dense/Relu' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "/Users/lukas/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/lukas/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/Users/lukas/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/Users/lukas/Library/Python/3.11/lib/python/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 607, in run_forever
      self._run_once()

In [ ]:
# Train the model
history = model.fit(trainGenerator, epochs=25, steps_per_epoch=20, validation_data = validationGenerator, verbose = 1, validation_steps=3)